In [34]:
pip install speechRecognition


Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install pyttsx3


Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install pywhatkit


Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install pyaudio


Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install wikipedia


Note: you may need to restart the kernel to use updated packages.


In [87]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import speech_recognition as sr
import pyttsx3
import pywhatkit
import pyaudio as pa 
import wikipedia
import datetime                           
import os            
import webbrowser

In [88]:
class HyperlinkManager:
      
    def __init__(self, text):
        self.text = text
        self.text.tag_config("hyper", foreground="blue", underline=1)
        self.text.tag_bind("hyper", "<Enter>", self._enter)
        self.text.tag_bind("hyper", "<Leave>", self._leave)
        self.text.tag_bind("hyper", "<Button-1>", self._click)

        self.reset()

    def reset(self):
        self.links = {}

    def add(self, action):
        # add an action to the manager.  returns tags to use in
        # associated text widget
        tag = "hyper-%d" % len(self.links)
        self.links[tag] = action
        return "hyper", tag

    def _enter(self, event):
        self.text.config(cursor="hand2")

    def _leave(self, event):
        self.text.config(cursor="")

    def _click(self, event):
        for tag in self.text.tag_names(CURRENT):
            if tag[:6] == "hyper-":
                self.links[tag]()
                return


In [89]:
# Importing the dataset
training_dataset = pd.read_csv(r'C:\Users\acer\Downloads\Training.csv')
test_dataset = pd.read_csv(r'C:\Users\acer\Downloads\Testing.csv')


In [90]:
X = training_dataset.iloc[:, 0:132].values
print(X)

[[1 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 1 1 1]]


In [91]:
X.shape

(4920, 132)

In [92]:
print("Shape of the array:", X.shape)
print("Number of dimensions:", X.ndim)
print("Total number of elements:", X.size)


Shape of the array: (4920, 132)
Number of dimensions: 2
Total number of elements: 649440


In [93]:
print("Data type of elements:", X.dtype)

Data type of elements: int64


In [94]:
print("Array summary:")
print(X)


Array summary:
[[1 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 1 1 1]]


In [95]:
y = training_dataset.iloc[:, -1].values
print(y)

['Fungal infection' 'Fungal infection' 'Fungal infection' ...
 'Urinary tract infection' 'Psoriasis' 'Impetigo']


In [96]:
# Dimensionality Reduction for removing redundancies
dimensionality_reduction = training_dataset.groupby(training_dataset['prognosis']).max()
print(dimensionality_reduction)

                                         itching  skin_rash  \
prognosis                                                     
(vertigo) Paroymsal  Positional Vertigo        0          0   
AIDS                                           0          0   
Acne                                           0          1   
Alcoholic hepatitis                            0          0   
Allergy                                        0          0   
Arthritis                                      0          0   
Bronchial Asthma                               0          0   
Cervical spondylosis                           0          0   
Chicken pox                                    1          1   
Chronic cholestasis                            1          0   
Common Cold                                    0          0   
Dengue                                         0          1   
Diabetes                                       0          0   
Dimorphic hemmorhoids(piles)                   0       

In [97]:
# Encoding String values to integer constants
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)
print(y)

[15 15 15 ... 38 35 27]


In [98]:
# Splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [99]:
# Implementing the Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier()

In [100]:
# Saving the information of columns
cols     = training_dataset.columns
cols     = cols[:-1]

In [101]:
# Checking the Important features
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

In [102]:
from sklearn.tree import _tree

In [103]:
def execute_bot():

    print("Please reply with yes/Yes or no/No for the following symptoms") 
    def print_disease(node):
        #print(node)
        node = node[0]
        #print(len(node))
        val  = node.nonzero() 
        #print(val)
        disease = labelencoder.inverse_transform(val[0])
        return disease
    def tree_to_code(tree, feature_names):
        tree_ = tree.tree_
        #print(tree_)
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]
        #print("def tree({}):".format(", ".join(feature_names)))
        symptoms_present = []
        def recurse(node, depth):
            indent = "  " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                print(name + " ?")
                ans = input()
                ans = ans.lower()
                if ans == 'yes':
                    val = 1
                else:
                    val = 0
                if  val <= threshold:
                    recurse(tree_.children_left[node], depth + 1)
                else:
                    symptoms_present.append(name)
                    recurse(tree_.children_right[node], depth + 1)
            else:
                present_disease = print_disease(tree_.value[node])
                print( "You may have " +  present_disease )
                print()
                red_cols = dimensionality_reduction.columns 
                symptoms_given = red_cols[dimensionality_reduction.loc[present_disease].values[0].nonzero()]
                print("symptoms present  " + str(list(symptoms_present)))
                print()
                print("symptoms given "  +  str(list(symptoms_given)) )  
                print()
                confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
                print("confidence level is " + str(confidence_level))
                print()
                print('The model suggests:')
                print()
                row = doctors[doctors['disease'] == present_disease[1]]
                print('Consult ', str(row['name'].values))
                print()
                print('Visit ', str(row['link'].values))
                print(present_disease[0])
                
    
        recurse(0, 1)
    
    tree_to_code(classifier,cols)


In [112]:
doc_dataset = pd.read_csv(r'C:\Users\acer\Downloads\Training.csv', names=['Name', 'Description'])

diseases = dimensionality_reduction.index
doctors = pd.DataFrame({'disease': diseases, 'name': np.nan, 'link': np.nan})

for disease in diseases:
    record = doc_dataset[doc_dataset['Name'] == disease]
    if not record.empty:
        doctors.loc[doctors['disease'] == disease, ['name', 'link']] = record.values[0]

# Now you can access the information for a specific disease like 'AIDS'
record = doctors[doctors['disease'] == 'AIDS']
doctor_name = record['name'].values[0]
doctor_link = record['link'].values[0]
print("Doctor Name:", doctor_name)
print("Doctor Link:", doctor_link)

C:\Users\acer\AppData\Local\Temp\ipykernel_18480\3674935527.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  doc_dataset = pd.read_csv(r'C:\Users\acer\Downloads\Training.csv', names=['Name', 'Description'])


Doctor Name: nan
Doctor Link: nan


In [113]:
listener = sr.Recognizer() 
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

In [114]:
def talk(text):
    engine.say(text)
    engine.runAndWait()

In [115]:
def take_command():
    try:
        with sr.Microphone() as source:
            print('listening...')
            voice = listener.listen(source)
            command = listener.recognize_google(voice)
            command = command.lower()
        
        if 'alexa' in command:
            command = command.replace('alexa', "")
            print(command)    
            
    except:
      pass  
    return command

In [116]:
def run_alexa():
    command = take_command()
    print(command)
    if 'what' in command:
        person = command.replace('what','')
        info = wikipedia.summary(person,1)
        print(info)
        talk(info)
    elif 'give more information' in command:
        person = command.replace('give more information','')
        info = wikipedia.summary(person,1)
        print(info)
        talk(info)
        
#    elif 'consult a doctor' in command:
#       person = command.replace('consult a doctor','')
#       info = (person,"Dr.Sunil Pathak")
#       print(info)
#        talk(info)  
        
    else:
        talk('please say the command again.')

In [117]:
# Execute the bot and see it in Action
execute_bot()

Please reply with yes/Yes or no/No for the following symptoms
slurred_speech ?


 no


pain_behind_the_eyes ?


 no


receiving_blood_transfusion ?


...

 no


rusty_sputum ?


 yes


['You may have Pneumonia']

symptoms present  ['rusty_sputum']

symptoms given ['chills', 'fatigue', 'cough', 'high_fever', 'breathlessness', 'sweating', 'malaise', 'phlegm', 'chest_pain', 'fast_heart_rate', 'rusty_sputum']

confidence level is 0.09090909090909091

The model suggests:



IndexError: index 1 is out of bounds for axis 0 with size 1

In [62]:
while True:
    run_alexa()

listening...
what is chickenpox
Chickenpox, or chicken pox, also known as varicella, is a highly contagious, vaccine-preventable disease caused by the initial infection with varicella zoster virus (VZV), a member of the herpesvirus family.
listening...


UnboundLocalError: cannot access local variable 'command' where it is not associated with a value